<a href="https://colab.research.google.com/github/PrasadSingh/coursera-test/blob/master/CrowdSize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import cv2
import numpy as np

In [25]:
# Download the weights and cfg files using the following commands
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg


net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
#The line below has been changed to accomodate the changes in the OpenCV library.
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

--2024-09-20 02:50:41--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.2’

yolov3.weights.2    100%[===================>] 236.52M  30.6MB/s    in 9.1s    

2024-09-20 02:50:50 (26.0 MB/s) - ‘yolov3.weights.2’ saved [248007048/248007048]

--2024-09-20 02:50:50--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg.2’

yolov3.cfg.2        100%[===================>]   8.15K  --.-KB/s    in 0s   

In [31]:
video_path = 'https://www.youtube.com/watch?v=y2zyucfCyjM'
cap = cv2.VideoCapture(video_path)

In [27]:
# Define time frame
start_frame = 14 * 30 # Assuming 30 FPS
end_frame = 32 * 30

In [28]:
# Set the current frame position to start frame
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

False

In [29]:
frame_count = start_frame
people_count = 0

In [30]:

people_count = 0
boxes = []
confidences = []
class_ids = []
end_frame = 30  # Define your end frame based on the video

while cap.isOpened() and frame_count <= end_frame:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Forward pass to get output
    outs = net.forward(output_layers)

    # Process the output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # class_id 0 for 'person'
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maxima Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in indices:
        i = i[0]  # NMS returns a list of lists
        people_count += 1

    frame_count += 1

cap.release()
cv2.destroyAllWindows()

print(f"Estimated number of people on the bridge: {people_count}")

Estimated number of people on the bridge: 0
